## Importación de librerías para los sistemas de recomendación

In [41]:
import os
import numpy as np  
import pandas as pd

from surprise import ( 
  Dataset,
  Reader,
  accuracy, 
  SVD,
  AlgoBase,
  BaselineOnly
) 

from surprise.model_selection import (
  train_test_split
)

## Clase `DataLoader`

La clase `DataLoader` está diseñada para cargar y manipular datos relacionados para un sistema de recomendación. A continuación se detalla su funcionalidad:

- Constructor (`__init__`)
  - Inicializa la instancia con tres argumentos que representan las rutas de acceso a los archivos de datos de usuarios, items y datos de interacción (ratings). Construye las rutas completas combinándolas con la ruta actual del directorio de trabajo del sistema operativo
  - Carga automáticamente los conjuntos de datos para usuarios, items y datos de interacción (ratings) utilizando el método `load_set`
- Método `load_set`
  - Recibe como argumento `name` que indica qué conjunto de datos cargar ( 'DATA', 'USER', 'ITEM' )
  - Dependiendo del valor de `name`, carga un DataFrame correspondiente desde un archivo CSV, especificando las columnas esperadas y ajustando el separador y la codificación según sea necesario
  - Para los conjuntos de datos 'DATA' y 'USER', elimina columnas no deseadas después de la carga
  - Para el conjunto de datos 'ITEM', también elimina columnas específicas no necesarias
- Método `load_dataset`
  - Crea un objeto `Dataset` de la biblioteca `surprise` a partir del conjunto de datos de interacción cargado previamente, preparándolo para su uso en modelos de recomendación.
  - Configura el `Reader` para interpretar las escalas de calificación de los ratings.
- Métodos de consulta
  - `get_user_by_id`: Busca información de un usuario por su ID, devolviendo un diccionario con detalles del usuario.
  - `get_item_by_id`: Busca información de un ítem por su ID, devolviendo un diccionario con detalles del ítem.
  - `get_rating_by_ids`: Intenta encontrar el rating dado un ID de usuario e ID de ítem, devolviendo el rating junto con un indicador de éxito (True si se encontró, False en caso contrario).
  - `get_ratings_by_name_id`: Filtra los datos de interacción para encontrar ratings basados en un criterio específico (por ejemplo, ID de género), devolviendo un DataFrame con los ratings encontrados.

In [140]:
class DataLoader: 
  
  def __init__(self, data_path: str, item_path: str, user_path: str) -> None:
    
    current = os.getcwd () [ 0 : os.getcwd ().rfind( '\\' ) ]
    self.DATA_PATH = current + data_path
    self.ITEM_PATH = current + item_path
    self.USER_PATH = current + user_path

    self.data_set = self.load_set ( 'DATA' )
    self.item_set = self.load_set ( 'ITEM' )
    self.user_set = self.load_set ( 'USER' )

  def load_set (self, name: str ) -> pd.DataFrame:

    if name == 'DATA':
      columns = [ 'userID', 'itemID', 'rating', 'timestamp' ]
      df = pd.read_csv ( 
        self.DATA_PATH, 
        names=columns, 
        sep='\t', 
        encoding='latin-1', 
        skipinitialspace=True 
      )
      df = df.drop ( columns= [ 'timestamp' ] )
      return df
  
    if name == 'USER':
      columns = [ 'userID', 'age', 'gender', 'occupation', 'zipCode' ]
      df = pd.read_csv ( 
        self.USER_PATH, 
        names=columns, 
        sep='|', 
        encoding='latin-1', 
        skipinitialspace=True 
      )
      df = df.drop ( columns= [ 'zipCode' ] )
      return df
  
    if name == 'ITEM':
      columns = [ 
        'itemID', 
        'name', 
        'releaseDate', 
        'videoReleaseDate', 
        'IMDbURL', 
        'gender_unknown', 
        'gender_action', 
        'gender_adventure', 
        'gender_animation', 
        'gender_children', 
        'gender_comedy',
        'gender_crime',
        'gender_documentary',
        'gender_drama',
        'gender_fantasy',
        'gender_film_noir',
        'gender_horror',
        'gender_musical',
        'gender_mystery',
        'gender_romance',
        'gender_scifi',
        'gender_thriller',
        'gender_war',
        'gender_western',
      ]
      df = pd.read_csv ( 
        self.ITEM_PATH, 
        names=columns, 
        sep='|', 
        encoding='latin-1', 
        skipinitialspace=True 
      )
      df = df.drop ( columns= [ 'videoReleaseDate', 'IMDbURL' ] )
      return df

  def load_dataset ( self ) -> Dataset:
    reader = Reader ( rating_scale= ( 1,5 ) )
    data = Dataset.load_from_df ( self.data_set [ [ 'userID', 'itemID', 'rating' ] ], reader )
    return data

  def get_user_by_id ( self, id: int ):
    info = self.user_set.loc [ self.user_set[ 'userID' ] == id ]
    return info[ [ 'userID', 'age', 'gender', 'occupation' ] ].iloc[0].to_dict()

  def get_item_by_id ( self, id: int ):
    info = self.item_set.loc [ self.item_set[ 'itemID' ] == id ]
    return info[ [ 
      'itemID', 
      'name', 
      'releaseDate', 
      'gender_unknown', 
      'gender_action', 
      'gender_adventure', 
      'gender_animation', 
      'gender_children', 
      'gender_comedy',
      'gender_crime',
      'gender_documentary',
      'gender_drama',
      'gender_fantasy',
      'gender_film_noir',
      'gender_horror',
      'gender_musical',
      'gender_mystery',
      'gender_romance',
      'gender_scifi',
      'gender_thriller',
      'gender_war',
      'gender_western', ] ].iloc[0].to_dict()

  def get_rating_by_ids ( self, user_id: int, item_id: int ):
    try:
      rating = self.data_set.loc [ self.data_set[ 'userID' ] == user_id ].loc [ self.data_set[ 'itemID' ] == item_id ]
      return ( rating.iloc[0]['rating'], True )
    except:
      # Failed to retrieve the rating
      return ( -1, False )

  def get_ratings_by_name_id ( self, column_name: str, id: int ):
    filtered_data = self.data_set.loc [ self.data_set[ column_name ] == id ]
    return filtered_data [ [ 'userID', 'itemID', 'rating' ] ]


Las direcciones donde tienen los datos para el sistema de recomendación son:

In [80]:
DATA_PATH = '\\dataset\\data.csv'
ITEM_PATH = '\\dataset\\item.csv'
USER_PATH = '\\dataset\\user.csv'

In [141]:
loader = DataLoader( 
  data_path=DATA_PATH,
  item_path=ITEM_PATH,
  user_path=USER_PATH 
)

In [142]:
loader.data_set
#loader.item_set
#loader.user_set

,userID,itemID,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [143]:
ratings_by_user_id = loader.get_ratings_by_name_id ( column_name='userID', id=196 )

results = f"""
========================================

Results Dataframe sort values by item id
{ ratings_by_user_id.sort_values ( by='itemID', ascending=True ) }

========================================

Results Dataframe sort values by rating
{ ratings_by_user_id.sort_values ( by='rating', ascending=False ) }

========================================

Shape: { ratings_by_user_id.shape }

"""
print ( results )

rating_by_ids = loader.get_rating_by_ids ( item_id=242, user_id=196 )
print ( f'Rating: { rating_by_ids }' )

info_item = loader.get_item_by_id ( id=242 )
print ( info_item )

info_user = loader.get_user_by_id ( id=196 )
print ( info_user )




Results Dataframe sort values by item id
       userID  itemID  rating
17102     196       8       5
56628     196      13       2
10981     196      25       4
22271     196      66       3
2374      196      67       5
21605     196      70       3
14606     196      94       3
23189     196     108       4
87863     196     110       1
10017     196     111       4
33536     196     116       3
52726     196     153       5
59607     196     173       2
24030     196     202       3
7517      196     238       4
0         196     242       3
1812      196     251       3
22773     196     257       2
78787     196     269       3
36281     196     285       5
13733     196     286       5
32721     196     287       3
6910      196     306       4
25726     196     340       3
1133      196     381       4
35197     196     382       4
940       196     393       4
50147     196     411       4
17830     196     428       4
10254     196     580       2
1896      196     655      

## Clase `Exploratory_Data_Analysis`

El Análisis Exploratorio de Datos permite determinar la mejor manera de manipular lsa fuentes de datos para obtener las respuesta que necesita, permitiendo descubrir patrones, detectar anomalías, probar una hipótesis o verificar suposiciones. 

El objetivo de tener una clase para el Análisis Exploratorio de Datos es ayudar a analizar los datos antes de hacer las predicciones. Permitiendo en algunos casos identificar errores obvios, así como comprender mejor los patrones dentro de los datos e incluso encontrar relaciones interesantes entre las variables. 

Herramientas que pudieran usarse para el Análisis Exploratorio de Datos: 
- Técnicas de clustering y reducción de dimensiones
- Visualización univariante de cada campo del conjunto de datos sin procesar, con estadísticas de resumen
- Visualización bivariante y estadísticas de resumen que le permiten evaluar la relación entre cada variable del conjunto de datos y la variable de destino que está viendo
- Visualización multivariantes, para mapear y comprender las interacciones entre los diferentes campos de datos 
- Métodos predictivos como la regresión lineal permite la predicción de resultados

In [85]:
class Exploratory_Data_Analysis: 
  pass



## Clase `Metrics`

La clase `Metrics` está diseñada para calcular y almacenar métricas de evaluación comunes en proyectos de sistemas de recomendación. 

- Constructor (`__init__`)
  - Inicializa la instancia con un argumento `predictions`, que representa las predicciones generadas por un modelo de recomendación
  - Inicializa un diccionario `metrics` vacío para almacenar los resultados de las métricas calculadas 
- Método `compute_metrics`
  - Permite calcular múltiples métricas de evaluación a la vez, pasando los nombres de las métricas como argumentos
  - Actualiza el diccionario `metrics` con los valores calculados para las métricas solicitadas
  - Retorna el diccionario `metrics` actualizado
- Método de Métricas
  - `MAE`: abreviatura de Mean Absolute Error, y calcula el Error Absoluto Medio entre las predicciones y los valores reales. Este método actualiza el diccionario `metrics` con el valor de MAE calculado.
  - `RMSE`: abreviatura de Root Mean Square Error, y calcula el Error Cuadrático Medio Raíz entre las predicciones y los valores reales. Este método actualiza el diccionario `metrics` con el valor de RMSE calculado.


In [46]:
class Metrics: 
  def __init__(self, predictions) -> None:
    self.predictions = predictions
    self.metrics = { }

  def compute_metrics ( self, *args ) -> dict :
    if 'MAE' in args: self.MAE ( )
    if 'RMSE' in args: self.RMSE ( )
    return self.metrics

  def MAE  ( self ):
    self.metrics [ 'MAE' ] = accuracy.mae ( self.predictions ) 

  def RMSE ( self ): 
    self.metrics [ 'RMSE' ] = accuracy.rmse ( self.predictions ) 

## Clase `DataGenerator`

La clase `DataGenerator` está diseñada para facilitar la división de un conjunto de datos en subconjuntos de entrenamiento y prueba, lo cual es una tarea común en el desarrollo y evaluación de modelos de Machine Learning y Sistemas de Recomendación. A continuación, se detalla su funcionalidad:
- Constructor (`__init__`)
  - `data`: conjunto de datos general que se desea dividir en entrenamiento y prueba
  - `percentage`: un valor opcional que especifica el tamaño del subconjunto de prueba como un porcentaje del total de datos. Por defecto este valor es 0.25, lo que significa que el 25% de los datos se reservará para el conjunto de prueba, dejando el 75% para el conjunto de entrenamiento
  - Utiliza la función `train_test_split` de scikit-learn para dividir el conjunto de datos en entrenamiento y prueba basándose en el tamaño especificado por `percentage`. Se establece un estado aleatorio fijo (`random_state = 1`) para garantizar la reproducibilidad de la división
  - Guarda los subconjuntos resultantes en los atributos `self.trainset` y `self.testset`
- Métodos 
  - `get_trainset`: Retorna el subconjunto de datos designado para entrenamiento
  - `get_testset`: Retorna el subconjunto de datos designado para prueba

La clase `DataGenerator` automatiza el proceso de preparación de los datos para el entrenamiento y la evaluación de modelos de Machine Learning. Al instanciar `DataGenerator` con un conjunto de datos y un porcentaje para la división de prueba, se genera automáticamente una partición de los datos en conjuntos de entrenamiento y prueba, facilitando la evaluación de modelos sin tener que repetir manualmente el proceso de división cada vez que se necesita entrenar o probar un modelo

In [47]:
class DataGenerator: 
  def __init__(self, data, percentage = 0.25) -> None:
    """Data Generator

    Build a 75/25 train/test split for measuring accuracy

    Args:
        percentage (float, optional): _description_. Defaults to 0.25.
    """
    self.trainset, self.testset = train_test_split ( data, test_size=percentage, random_state=1 )

# MEJORAR CON UN METODO PROPIO PARA EL TRAIN-TEST SPLIT con los DATAFRAME y RANDOM 

  def get_trainset( self ):
    return self.trainset
  
  def get_testset( self ):
    return self.testset

## Clase `Model`

La clase `Model` está diseñada para encapsular un modelo de recomendación, proporcionando una interfaz simplificada para su evaluación mediante el uso de un generador de datos, que no es más que un objeto que contiene el conjunto de datos de entrenamiento y el conjunto de datos de prueba, y el cálculo de métricas de rendimiento específicas. A continuación, se detalla su funcionalidad:

- Constructor (`__init__`)
  - Inicializa la instancia con dos argumentos: `model` (objeto que representa el modelo de recomendación que se va a evaluar. Este modelo debe tener métodos `fit` para entrenamiento y `test` para generar predicciones) y `name` (cadena que indica el modelo, útil para referencias e informes de evaluación)
  - Almacena estos dos argumentos como atributos de la instancia
- Método `__str__`
  - Proporciona una representación en forma de cadena del objeto `Model`, retornando una cadena que incluye el nombre del modelo. Esto es útil para imprimir información sobre el modelo de manera legible por humanos, facilitando la identificación del modelo cuando se imprime el objeto
- Método `evaluate`
  - Este método evalúa el rendimiento utilizando datos proporcionados por un objeto `DataGenerator`. 
    - Utiliza el conjunto de entrenamiento y prueba contenidos dentro de un objeto `DataGenerator` para entrenar el modelo y generar predicciones
    - Primero, entrena el modelo con el conjunto de entrenamiento obtenido mediante `data.get_trainset()`
    - Luego, utiliza el conjunto de prueba obtenido mediante `data.get_testset()` para generar predicciones. 
    - Calcula métricas de rendimiento específicas usando las predicciones generadas y los compara con los valores reales
    - Retorna un diccionario con las métricas calculadas 



In [48]:
class Model: 
  def __init__(self, model, name) -> None:
    self.model = model
    self.name = name
  
  def __str__(self) -> str:
    return f'Model: { self.name }'
  
  def evaluate ( self, data: DataGenerator ): 
    trainset = data.get_trainset ( )
    testset = data.get_testset ( )

    fit_model = self.model.fit ( trainset )
    predictions = fit_model.test ( testset )

    metrics = Metrics ( predictions ).compute_metrics( 'MAE', 'RMSE' )
    
    return metrics

## Clase `Factory`

La clase `Factory` en el código proporcionado es una clase de diseño que actúa como un contenedor y administrador para una colección de modelos (`Model`). Esta clase está diseñada para manejar operaciones relacionadas con los modelos que pueden usarse para sistemas de recomendación. Aquí hay una descripción detallada de lo que hace cada método dentro de esta clase:

- Constructor `__init__(self, dataset) -> None`
  - **Propósito**: Inicializa una instancia de la clase `Factory`.
  - **Parámetros**:
    - `dataset`: Un parámetro que se pasa al constructor para indicar el conjunto de datos con el cual se trabajarán los modelos.
  - **Acciones**:
    - Crea una instancia de `DataGenerator` pasando el `dataset` proporcionado y la asigna a `self.dataset`. Esto sugiere que `DataGenerator` es una clase responsable de preparar o formatear el conjunto de datos de alguna manera antes de su uso.
    - Inicializa una lista vacía llamada `models`, destinada a almacenar instancias de modelos.

- Método `add_model(self, model: Model) -> None`
  - **Propósito**: Agrega un modelo a la lista de modelos manejados por la instancia de `Factory`.
  - **Parámetros**:
    - `model`: El modelo a agregar a la lista de modelos. Se espera que este argumento sea una instancia de la clase `Model` o una subclase de ella.
  - **Acciones**:
    - Añade el modelo pasado como argumento a la lista `self.models`.

- Método `evaluate(self) -> dict`
  - **Propósito**: Evalúa todos los modelos almacenados en la lista de modelos utilizando el mismo conjunto de datos.
  - **Acciones**:
    - Inicializa un diccionario vacío llamado `results`.
    - Itera sobre cada modelo en `self.models`.
    - Imprime un mensaje indicando que se está evaluando el modelo actual.
    - Llama al método `evaluate` del modelo actual, pasándole el conjunto de datos `self.dataset`, y guarda el resultado en el diccionario `results` bajo la clave correspondiente al nombre del modelo.
    - Devuelve el diccionario `results`, que contiene los resultados de la evaluación de todos los modelos.

- Método `clean_models(self) -> None`
  - **Propósito**: Limpia la lista de modelos, eliminando todos los modelos almacenados.
  - **Acciones**:
    - Asigna una lista vacía a `self.models`, efectivamente limpiando cualquier modelo previamente agregado.


In [49]:
class Factory:
  def __init__(self, dataset) -> None:
    self.dataset = DataGenerator ( dataset )
    self.models: list[ Model ] = [ ]
  
  def add_model ( self, model: Model ):
    self.models.append ( model )
  
  def evaluate ( self ):
    results = { }
    for model in self.models:
      print ( f'Evaluating { model.name }' )
      results [ model.name ] = model.evaluate( self.dataset )

  def clean_models ( self ):
    self.models = [] 


A partir de aquí se muestran algunas pruebas sobre las clases creadas

In [66]:
model_svd = Model ( model=SVD(), name='SVD' )
factory = Factory( loader.load_dataset( ) )

factory.add_model( model_svd )
factory.evaluate()

Evaluating SVD
MAE:  0.7402
RMSE: 0.9411


In [67]:
model_baseline = Model ( model=BaselineOnly(), name='Baseline Only' )
factory = Factory( loader.load_dataset( ) )

factory.add_model( model_baseline )
factory.evaluate()

Evaluating Baseline Only
Estimating biases using als...
MAE:  0.7501
RMSE: 0.9485


## Clase `HybridModel`

La clase `HybridModel` hereda de `AlgoBase` y está diseñada específicamente para sistemas de recomendación híbridos, combinando múltiples modelos individuales (`Model`) para generar recomendaciones basadas en pesos predefinidos. Este enfoque innovador permite integrar diversas técnicas de aprendizaje automático o modelos estadísticos, optimizando así la precisión y robustez de las recomendaciones generadas. A continuación, se detalla lo que cada componente de esta clase contribuye al funcionamiento general:

- Constructor `__init__(self, models, weights, **kwargs)`
  - **Propósito**: Inicializa una instancia de `HybridModel` en el contexto de un sistema de recomendación.
  - **Parámetros**:
    - `models`: Colección de modelos individuales (`Model`) seleccionados para colaborar en la generación de recomendaciones.
    - `weights`: Lista de pesos asociados a cada modelo en `models`, que determinan su influencia relativa en la recomendación final.
    - `**kwargs`: Parámetros adicionales que pueden ser pasados al constructor base `AlgoBase`, posiblemente incluyendo configuraciones específicas del sistema de recomendación.
  - **Acciones**:
    - Invoca el constructor de la clase base `AlgoBase` con cualquier argumento adicional proporcionado, asegurando la correcta inicialización dentro del marco de un sistema de recomendación.
    - Asigna la lista de modelos a `self.models`, estableciendo la base para la integración de modelos.
    - Asigna la lista de pesos a `self.weights`, configurando la estrategia de combinación ponderada utilizada para fusionar las predicciones de los modelos.

- Método `fit(self, trainset)`
  - **Propósito**: Entrena todos los modelos individuales contenidos en el modelo híbrido, preparándolos para la generación de recomendaciones.
  - **Parámetros**:
    - `trainset`: Conjunto de datos de entrenamiento utilizado para ajustar los modelos internos, asegurando que estén bien calibrados para el dominio de recomendación.
  - **Acciones**:
    - Ejecuta el método `fit` de la clase base `AlgoBase` para realizar cualquier configuración o ajuste inicial necesario, adaptándose al entorno del sistema de recomendación.
    - Procesa cada modelo en `self.models`, entrenándolos individualmente con el conjunto de datos de entrenamiento `trainset`, afinando sus capacidades predictivas.
    - Retorna `self`, facilitando la concatenación de operaciones, como la evaluación posterior o la generación de recomendaciones.

- Método `estimate(self, user_id, item_id)`
  - **Propósito**: Genera una recomendación para un usuario y un ítem específicos mediante la integración ponderada de las predicciones de los modelos individuales.
  - **Parámetros**:
    - `user_id`: Identificador único del usuario cuya preferencia se está estimando.
    - `item_id`: Identificador único del ítem o elemento para el cual se está generando una recomendación.
  - **Acciones**:
    - Inicializa variables `scores` y `weight` a 0, preparándose para calcular la puntuación ponderada de recomendación.
    - Itera sobre cada modelo en `self.models`, calcula la predicción de cada modelo para el par `(user_id, item_id)` multiplicada por su peso correspondiente, acumulando estos valores en `scores`.
    - Suma el peso correspondiente a cada modelo a `weight`, construyendo la base para el cálculo de la media ponderada.
    - Retorna la media ponderada de las predicciones de todos los modelos, dividiendo `scores` entre `weight`, produciendo una recomendación equilibrada que refleja la opinión de todos los modelos participantes.

In [72]:
class HybridModel ( AlgoBase ):

  def __init__ (self, models, weights, **kwargs):
    super().__init__(**kwargs) 
    self.models: list[ Model ] = models
    self.weights = weights
  
  def fit (self, trainset):
    AlgoBase.fit ( self, trainset )
    for model in self.models:
      model.model.fit ( trainset )
    return self
  
  def estimate ( self, user_id, item_id ):
    scores = 0 
    weight = 0
    for i in range ( len( self.models ) ):
      scores += self.models[i].model.predict ( user_id, item_id ).est * self.weights[i]
      weight += self.weights[i]

    return scores/weight

Aqui se tiene un ejemplo básico del uso del modelo híbrido de recomendación

In [71]:
models = [ 
  model_svd,
  model_baseline
]
weights = [ 
  0.5,
  0.5
]

hybrid = HybridModel ( models, weights )

factory = Factory ( loader.load_dataset() )

model = Model ( hybrid, 'Hybrid: SVD - Baseline' )
factory.add_model ( model )

factory.evaluate()

Evaluating Hybrid: SVD - Baseline
Estimating biases using als...
MAE:  0.9949
RMSE: 1.2280


TODO: 
- Completar la clase de EDA
- Mejorar los sistemas de recomendacion hibridos